In [ ]:
import openai
import langchain
import pinecone
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os

In [ ]:
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [ ]:
docs = read_doc(r"C:\Users\shubh\shubham\projects\rag_on_pdf\documents")

In [ ]:
# docs

In [ ]:
def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc

In [ ]:
chunked_docs = chunk_data(docs = docs)

In [ ]:
docs == chunked_docs

In [ ]:
len(chunked_docs)

In [ ]:
embeddings = OpenAIEmbeddings(api_key = os.environ['OPENAI_API_KEY'])

In [ ]:
pinecone.init(
    api_key = os.environ['PINECONE_API_KEY'],
    environment = os.environ['ENVIRONMENT_NAME']
)
index_name = "rag_vector_embeds"


In [ ]:
Pinecone.from_documents(chunked_docs, embeddings, index_name=index_name)

In [ ]:
def retrieve_query(query, k=2):
    matching_results = index.similarity_search(query, k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct", temperature=0.5)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
def retrieve_answer(query):
    doc_search = retrieve_query(query)
    response = chain.run(input_documents = doc_search, question=query)
    return response

In [ ]:
question = "How much increase was observed in mobile phone production this year?"
answer = retrieve_answer(question)
print(answer)